![BTS](https://github.com/vfp1/bts-dsf-2020/blob/main/Logo-BTS.jpg?raw=1)

# Session 4: Pandas and SQLite in depth session

### Victor F. Pajuelo Madrigal <victor.pajuelo@bts.tech> - Data Science Foundations (2020-10-29)

Open this notebook in Google Colaboratory: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vfp1/bts-dsf-2020/blob/main/Session_4/04_Pandas_&_SQLite_recap_through_exercises.ipynb)


# More Pandas!

Check a good cheatsheet for Pandas [here](https://www.w3resource.com/python-exercises/pandas/index.php)

[Some cool exercise resources for Pandas](https://www.machinelearningplus.com/python/101-pandas-exercises-python/)

## Groupby exersises

Import this [dataset](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user)

*   What is the mean age per occupation?
*   What is the male ratio per occupation, sort it from the most to the least. Do the same for women
*   For each occupation, calculate the minimum and maximum ages
* For each combination of occupation and gender, calculate the mean age
* For each occupation present the percentage of women and men



In [ ]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user', delimiter = '|')
data.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
# Mean age per occupation
data.groupby(['occupation']).age.mean().sort_values(ascending = False).head()

In [ ]:
# Male / Female ratio per occupation 
genderMF = 'M'
total = data.occupation.value_counts()
total_gender = data[data.gender == genderMF].occupation.value_counts()
gender_ratio = (total_gender/total)*100
print('Ratio of',genderMF,'per occupation')
gender_ratio.sort_values(ascending = False).head(10)

## Apply exercises

Import this [dataset](https://raw.githubusercontent.com/vfp1/bts-dsf-2020/main/data/US_Crime_Rates_1960_2014.csv)

* What is the type of the columns?
* Convert the type of the column Year to datetime64
* Set the Year column as the index of the dataframe
* Delete the Total column
* Group the year by decades and sum the values (take care with population values)
* What is the most dangerous decade to live in the US?


In [ ]:
crime = pd.read_csv('https://raw.githubusercontent.com/vfp1/bts-dsf-2020/main/data/US_Crime_Rates_1960_2014.csv', delimiter = ',', parse_dates = ['Year'])

In [ ]:
# Type of the columns
crime.dtypes

In [ ]:
# Convert column Year to datetime64
# NOTE: Because I imported with the "parse_dates" command above, 'Year' is already a datetime. But had I not done that, the command would be:
crime.Year = pd.to_datetime(crime.Year, format='%Y')
crime.dtypes

In [ ]:
# Set the Year column as the index of the datafram
crime.set_index('Year', inplace = 'True')

ValueError: ignored

In [ ]:
crime.index[1]

1

# SQLite and Pandas

## SQLite, Pandas and the world of tips

Given [this](https://github.com/vfp1/bts-dsf-2020/blob/main/data/tips.csv) CSV file. Do the following:

* Load it in Pandas
* Save the `DataFrame` as SQLite
* Reply the following questions by slicing the SQLite database with SQL queries and then using Pandas for analysis:
    * Who is most likely to receive a tip during weekday? A Female or a Male? 
        * And during weekends?
    * What is the table size that is most likely to bring a better tip?
    * Who are giving more tips, smokers or non smokers? 

In [1]:
import pandas as pd
import numpy as np

url = 'https://github.com/vfp1/bts-dsf-2020/raw/main/data/tips.csv'
# make sure this is "raw" and not "blob", otherwise it'll try to read the whole webpage

tips = pd.read_csv(url, error_bad_lines= False)
tips.head() 
import sqlite3

cnx = sqlite3.connect('tips.db')
tips.to_sql(name='tips', con=cnx) 
import sqlite3

def create_connect_database(db_name):
  """Opens connection or creates it if it is not existent"""
  connection = sqlite3.connect(db_name)
  cursor = connection.cursor()
  return connection, cursor

def commit_close(conn, c, close_connection=True):
  """Commits to connection DB and closes it if close is True"""
  conn.commit()

  if close_connection is True:
    c.close()
    conn.close()
  elif close_connection is False:
    pass 


In [2]:
# Who is more likely to receive a tip during the weekday / weekend?
conn, cursor = create_connect_database('tips.db')

query = """SELECT day, sex, COUNT(*)
FROM tips
GROUP BY day, sex;"""

#SELECT round(AVG(tip),5) tip_avg, * FROM tips GROUP BY size order by  tip_avg DESC;

#SELECT round(AVG(tip),5) tip_avg, smoker, size FROM tips GROUP BY size, smoker order by  tip_avg DESC;
df = pd.read_sql_query(query, conn)
df

,day,sex,COUNT(*)
0,Fri,Female,9
1,Fri,Male,10
2,Sat,Female,28
3,Sat,Male,59
4,Sun,Female,18
5,Sun,Male,58
6,Thur,Female,32
7,Thur,Male,30


## More airports, this time without spatial fears
 

Download [this Sqlite database](https://github.com/vfp1/bts-dsf-2020/raw/main/data/flights.db)

Using SQLite database reply the following questions:

*   Which is the country that has the least amount of active airlines?
*   And which is the country that has the most amount of active airlines?
*   Which is the country that has more airports?
*   And the country with the least airports?
*   Which country has the most outgoing destinations? And the least?
*   And which country has the most incoming destinations? And the least?

